In [ ]:
import sys
import itertools
from pathlib import Path
from os import getcwd
sys.path.append(str(Path(getcwd()).parent))
from os.path import join, basename

import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 1000)

from bulletin import default_input, default_output, hoje
from bulletin.systems import Notifica, CasosConfirmados
from bulletin.utils import utils, normalize
from bulletin.utils.static import Municipios
from bulletin.utils.xls_ import fit_cols

import random
import gc
from tqdm import tqdm

In [ ]:
notifica = Notifica('notifica')
notifica.load(compress=False)
# notifica.df

In [ ]:
cc = CasosConfirmados()
cc.load(f"cc_{hoje.strftime('%d_%m_%Y')}", compress=True)
# cc.df

In [ ]:
observed_cols = ['classificacao_final','evolucao','status_notificacao','excluir_ficha']

In [ ]:
# possiveis_alterados = cc.df.loc[cc.df['id_notifica'].isin(notifica.df['id'])]
possiveis_alterados = possiveis_alterados.drop_duplicates('id_notifica')
possiveis_alterados

In [ ]:
possiveis_atualizacoes = notifica.df.loc[notifica.df['id'].isin(cc.df['id_notifica'])]
possiveis_atualizacoes

In [ ]:
mergedf = pd.merge(possiveis_alterados[observed_cols+['id_notifica']].rename(columns={'id_notifica':'id'}),possiveis_atualizacoes[observed_cols+['id']],on='id',how='left',suffixes=['_old','_new'])
mergedf['updated'] = False
mergedf['updated_cols'] = ''
mergedf


In [ ]:
for col in observed_cols:
    print(f"{col} inferno")
    old_col = f'{col}_old'
    new_col = f'{col}_new'

    # criterios = ( ~(mergedf[old_col].isna() & mergedf[new_col].isna()) & (mergedf[old_col]!=mergedf[new_col]) )
    criterios = ( (mergedf[old_col] != mergedf[new_col]) )

    mergedf.loc[ criterios, 'updated' ] = True
    # mergedf.loc[ criterios, 'updated_cols' ] = mergedf.loc[ criterios,'updated_cols'].apply( lambda l: utils.strlist(l,col) )

    display(mergedf.loc[criterios,[old_col,new_col]])
    display(mergedf.loc[criterios,[old_col,new_col,'id']].groupby([old_col,new_col])[['id']].count())


In [ ]:
raise

In [ ]:
for col in observed_cols:
    old_col = f'{col}_old'
    new_col = f'{col}_new'

    criterios = ( ~(mergedf[old_col].isna() & mergedf[new_col].isna()) & (mergedf[old_col]!=mergedf[new_col]) )
    # criterios = ( (mergedf[old_col]!=mergedf[new_col]) )

    mergedf.loc[ criterios, 'updated_cols' ] = mergedf.loc[ criterios,'updated_cols'].apply( lambda l: utils.strlist(l,col) )
    mergedf.loc[ criterios, 'updated' ] = True


notifica.df = notifica.df.set_index('id')
notifica.df = notifica.df.append(novas_notificacoes)

mergedf = mergedf.loc[mergedf['updated']==True]

updated_cols = mergedf[['updated_cols','id']].groupby('updated_cols').count().sort_values('id',ascending=False).reset_index()
updated_cols.iloc[0:-1,0] += ','
updated_cols = list(set(updated_cols.sum()[0].split(',')))

for col in updated_cols:
    atualizacoes = possiveis_atualizacoes.loc[
        (
            possiveis_atualizacoes['id'].isin(mergedf.loc[mergedf['updated_cols'].str.contains(col),'id'])
        ),
        ['id',col]
    ].copy().set_index('id')

    print(f"atualizacoes {col}: {len(atualizacoes)}")
    notifica.df.loc[atualizacoes.index,[col]].update(atualizacoes)

notifica.df.reset_index(inplace=True)

mergedf.to_pickle(join(notifica.database_dir,f"updates_{hoje.strftime('%d_%m_%Y_%H%M')}.pkl"),'bz2')
return novas_notificacoes, mergedf
